In [1]:
import os
import json
#!pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
cid ='d8762235995b4ddcbc0df90739df9a63' # Client ID; copy this from your app created on beta.developer.spotify.com
secret = '887a4eccdbde4195bfc0f7f4c472fe70' # Client Secret; copy this from your app
username = '12155311349' # Your Spotify username

#for avaliable scopes see https://developer.spotify.com/web-api/using-scopes/
scope = 'user-library-read playlist-modify-public playlist-read-private'
redirect_uri='https://localhost:8080/callback/'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

token = util.prompt_for_user_token(username,scope,client_id=cid,client_secret=secret,redirect_uri="https://localhost:8080/callback/")
if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [3]:
# song_name = input("Search a song for recommendations: ")

# User History Input


In [4]:
short_term_tracks = sp.current_user_top_tracks(time_range='short_term', limit=50)
medium_term_tracks = sp.current_user_top_tracks(time_range='medium_term', limit=50)

# Merge the results and remove duplicates
all_tracks = short_term_tracks['items'] + medium_term_tracks['items']
unique_tracks = {track['id']: track for track in all_tracks}.values()

# Get the audio features for each track
audio_features = {}
for track in unique_tracks:
    audio_features[track['id']] = sp.audio_features(track['id'])[0]

# Create a dataframe with the track information
columns = ['song name', 'artist', 'popularity', 'release date', 'iD', 'danceability', 'energy','loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
data = []
for i, track in enumerate(unique_tracks):
    name = track['name']
    artist = track['artists'][0]['name']
    popularity = track['popularity']
    year = track['album']['release_date'][:4]
    track_id = track['id']
    audio_features = sp.audio_features([track_id])[0]
    if audio_features is None:
        danceability = 0
        loudness = 0
        energy = 0
        speechiness = 0
        acousticness = 0
        instrumentalness = 0
        liveness = 0
        valence = 0
        tempo = 0
    else:
        danceability = audio_features['danceability']
        loudness = audio_features['loudness']
        energy = audio_features['energy']
        speechiness = audio_features['speechiness']
        acousticness = audio_features['acousticness']
        instrumentalness = audio_features['instrumentalness']
        liveness = audio_features['liveness']
        valence = audio_features['valence']
        tempo = audio_features['tempo']
    data.append([name, artist, popularity, year, track_id, danceability, loudness,energy, speechiness, acousticness, instrumentalness, liveness, valence, tempo])
df = pd.DataFrame(data, columns=columns)

# Scale between 0 and 1
df["tempo"] = (df["tempo"] / df["tempo"].max())
df["loudness"] = (df["loudness"] / df["loudness"].max())
df["energy"] = (df["energy"] / df["energy"].min())
df["popularity"] = (df["popularity"] / 100)

In [5]:
df

# Getting Recommendations 

In [6]:
recom = []

for index, row in df.iterrows():
    # Find similar songs based on audio features, year, and popularity
    similar_songs = sp.recommendations(seed_tracks=[row['iD']], target_popularity=popularity, limit=5, **audio_features)
    recom.append(similar_songs)
        
# Extract relevant info from recommendations and store in DataFrame
columns = ['name', 'artist', 'popularity', 'year', 'iD', 'danceability', 'energy','loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
data = []
for i, track_list in enumerate(recom):
    for track in track_list['tracks']:
        name = track['name']
        artist = track['artists'][0]['name']
        popularity = track['popularity']
        year = track['album']['release_date'][:4]
        track_id = track['id']
        audio_features = sp.audio_features([track_id])[0]
        if audio_features is None:
            danceability = 0
            loudness = 0
            energy = 0
            speechiness = 0
            acousticness = 0
            instrumentalness = 0
            liveness = 0
            valence = 0
            tempo = 0
        else:
            danceability = audio_features['danceability']
            loudness = audio_features['loudness']
            energy = audio_features['energy']
            speechiness = audio_features['speechiness']
            acousticness = audio_features['acousticness']
            instrumentalness = audio_features['instrumentalness']
            liveness = audio_features['liveness']
            valence = audio_features['valence']
            tempo = audio_features['tempo']
        data.append([name, artist, popularity, year, track_id, danceability, loudness,energy, speechiness, acousticness, instrumentalness, liveness, valence, tempo])
df_recom = pd.DataFrame(data, columns=columns)

# Scale between 0 and 1
df_recom["tempo"] = (df_recom["tempo"] / df_recom["tempo"].max())
df_recom["loudness"] = (df_recom["loudness"] / df_recom["loudness"].max())
df_recom["energy"] = (df_recom["energy"] / df_recom["energy"].min())
df_recom["popularity"] = (df_recom["popularity"] / 100)

df_recom


In [7]:
df_recom = df_recom.drop_duplicates(subset=['name'])

In [8]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Define the features to use for the model
features = ['year','danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']

# Create a new dataframe with only the features to use for the model
df_model = df_recom[features]

# Scale the features
scaler = StandardScaler()
df_model_scaled = scaler.fit_transform(df_model)

# Fit the K-means model using the new dataframeabs
kmeans = KMeans(n_clusters=4, random_state=42)
kmeans.fit(df_model_scaled)

# Get the cluster labels for each data point
labels = kmeans.labels_
df_recom['clusters'] = labels
# Apply PCA to reduce the dimensionality of the data
pca = PCA(n_components=2)
pca_data = pca.fit_transform(df_model_scaled)

# Plot the data points with different colors based on their cluster labels
plt.scatter(pca_data[:, 0], pca_data[:, 1], c=labels, cmap='viridis')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('K-Means Clustering Results')
plt.legend()
plt.show()

In [9]:
df_recom

In [10]:
# filter the data to only include cluster 2 values
cluster_0_data = df_recom[df_recom['clusters']==0]
cluster_1_data = df_recom[df_recom['clusters']==1]
cluster_2_data = df_recom[df_recom['clusters'] == 2]
cluster_3_data = df_recom[df_recom['clusters']==3]

In [11]:
cluster_0_data.head()

In [12]:
cluster_1_data.head()

In [13]:
cluster_2_data.head()

In [14]:
cluster_3_data.head()

In [15]:
user_id = username

# Define a dictionary to hold the cluster playlists
cluster_playlists = {}

# Loop through the clusters and create a new playlist for each one
for cluster_id in range(4):
    playlist_name = f'Cluster {cluster_id}'
    playlist = sp.user_playlist_create(user=user_id, name=playlist_name)
    cluster_playlists[cluster_id] = playlist['id']

# Loop through the songs in each cluster and add them to their corresponding playlist
for cluster_id, cluster_data in enumerate([cluster_0_data, cluster_1_data, cluster_2_data, cluster_3_data]):
    for _, song_data in cluster_data.iterrows():
        song_id = song_data['iD']
        track_info = sp.track(song_id)
        track_uri = track_info['uri']
        sp.user_playlist_add_tracks(user=user_id, playlist_id=cluster_playlists[cluster_id], tracks=[track_uri])


In [16]:
# Define a list to hold the playlist information
playlist_info = []

# Loop through the cluster playlists and add the playlist ID and name to the list
for cluster_id in range(4):
    playlist_id = cluster_playlists[cluster_id]
    playlist_name = f'Cluster {cluster_id} Playlist'
    playlist_info.append({'Name': playlist_name, 'id': playlist_id})

# Create a DataFrame from the playlist information
playlist_df = pd.DataFrame(playlist_info)

# Display the playlist IDs and names
print(playlist_df)

In [ ]:
while True:
        action = input("Do you want to keep all playlists, delete all playlists, or keep a specific playlist? (Enter 'all', 'none', or a cluster number): ")
        if action == 'all':
            print("All playlists will be kept.")
        elif action == 'none':
            for playlist_id in playlist_df['id']:
                sp.user_playlist_unfollow(user=user_id, playlist_id=playlist_id)
            print("All playlists have been deleted.")
        elif action.isdigit() and 0 <= int(action) < 4:
            cluster_id = int(action)
            playlist_id = cluster_playlists[cluster_id]
            print(f"Playlist '{playlist_df.iloc[cluster_id]['Name']}' will be kept.")
            playlist_df = playlist_df.iloc[[cluster_id]]
        else:
            print("Invalid input. Please enter 'all', 'none', or a cluster number (0-3).")